In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  #交叉验证
from sklearn.metrics import confusion_matrix

import xgboost as xgb
import pandas as pd
import numpy as np
import pickle
import warnings

warnings.filterwarnings("ignore")

In [2]:
disorder_R_H = pickle.load(open('/home/dldx/R-H/code/classification/ML/data/disorder/avg_R-H_all_dsiorder.pkl','rb'))

data1 = pd.read_csv('/home/dldx/R-H/code/classification/ML/data/disorder/MSA_disorder_R_H.csv')
input_lable = np.array(data1.Tier)

In [3]:
all_21_IDR_disorder = []
for position in range(21):
    one_IDR = []
    R_IDR = [i[position] for i in disorder_R_H[0]]
    H_IDR = [i[position] for i in disorder_R_H[1]]
 
    # 转化为二维数组
    for i in range(72):
        test =[]
        test.append(R_IDR[i])
        one_IDR.append(test)
    for i in range(107):
        test =[]
        test.append(H_IDR[i])
        one_IDR.append(test)

    # 所有IDR的平均disorder值添加到一个数组
    all_21_IDR_disorder.append(one_IDR)

In [4]:
disorder_R_H[0][65][15]

0.03288253745995458

In [5]:
# 重新跑关于IDR16由数据更改的情况

# 评估结果
all_21_IDR_length_evaluate = []

# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

# 21个IDR区域分别进行分类
one_evaluate = []
for r in range(0,31):
    one_random_state = []
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR_disorder[15],input_lable,train_size=0.8,random_state = r)
    # 定义模型
    model = xgb.XGBClassifier()
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 最优的模型参数
    params = model.best_params_ 
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    one_random_state.append(accuracy)
    one_random_state.append(mcc)
    one_random_state.append(sens)
    one_random_state.append(spec)
    one_evaluate.append(one_random_state)

    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("最优的模型参数是：" + str(params))
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")
all_21_IDR_length_evaluate.append(one_evaluate)
print("以上为IDR__" + str(15 + 1) + "时，evaluation")

数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.75
  mcc : 0.5905854796650598
  sens : 53.333333333333336
  spec : 90.47619047619048
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.1746842814444709
  sens : 47.05882352941176
  spec : 68.42105263157895
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.5103103630798288
  sens : 50.0
  spec : 87.5
数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6944444444444444
  mcc : 0.41604060675337073
  sens : 56.25
  spec : 80.0
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6944444444444444
  mcc : 0.2645751311064591
  sens : 36.36363636363637
  spec : 84.0
数据集

In [ ]:
# 评估结果
all_21_IDR_length_evaluate = []

# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}

# 21个IDR区域分别进行分类
for position in range(21):
    one_evaluate = []
    for r in range(0,31):
        one_random_state = []
        # train_test_split 按比例划分原数据集
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR_disorder[position],input_lable,train_size=0.8,random_state = r)
        # 定义模型
        model = xgb.XGBClassifier()
        # 将模型加入超参数网格搜索和交叉验证     
        model = GridSearchCV(model, param_grid=param_dict, cv=5)
        # 训练数据集
        model = model.fit(Xtrain, Ytrain)
        # 最优的模型参数
        params = model.best_params_ 
        # 模型评估
        predict = model.predict(Xtest)
        cm = confusion_matrix(Ytest,predict)
        tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
        n = tp + fp + tn + fn
        accuracy = (tp + tn)/n 
        mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
        sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
        spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
        one_random_state.append(accuracy)
        one_random_state.append(mcc)
        one_random_state.append(sens)
        one_random_state.append(spec)
        one_evaluate.append(one_random_state)
        
        print("数据集划分 random_state：" + str(r) + " 时，")
        print("Evaluate：")
        print("最优的模型参数是：" + str(params))
        print("  accuracy : " + str(accuracy))
        print("  mcc : " + str(mcc))
        print("  sens : " + str(sens))
        print("  spec : " + str(spec)) 
        print("============================")
    all_21_IDR_length_evaluate.append(one_evaluate)
    print("以上为IDR__" + str(position + 1) + "时，evaluation")
    print("***********************************************************************")

数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.7222222222222222
  mcc : 0.48801455452776477
  sens : 53.333333333333336
  spec : 85.71428571428571
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5277777777777778
  mcc : 0.04499212706658476
  sens : 29.411764705882355
  spec : 73.68421052631578
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5
  mcc : 0.07844645405527362
  sens : 30.0
  spec : 75.0
数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.16262312563634834
  sens : 37.5
  spec : 75.0
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.0408248290463863
  sens : 36.36363636363637
  spec : 68.0
数据

数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.2531327526737512
  sens : 63.63636363636363
  spec : 68.0
数据集划分 random_state：5 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.4661384642714042
  sens : 42.10526315789473
  spec : 88.23529411764706
数据集划分 random_state：6 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : 0.04351941398892446
  sens : 35.714285714285715
  spec : 68.18181818181817
数据集划分 random_state：7 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.0
  sens : 25.0
  spec : 75.0
数据集划分 random_state：8 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.3424747597107866
  sens : 35.714285714285715
  spe

数据集划分 random_state：8 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.5
  mcc : -0.02383656473113981
  sens : 42.857142857142854
  spec : 54.54545454545454
数据集划分 random_state：9 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.3471874489966557
  sens : 52.63157894736842
  spec : 76.47058823529412
数据集划分 random_state：10 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.7222222222222222
  mcc : 0.5020790110464023
  sens : 56.25
  spec : 85.0
数据集划分 random_state：11 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.4330127018922193
  sens : 50.0
  spec : 83.33333333333334
数据集划分 random_state：12 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5277777777777778
  mcc : 0.04499212706658476
  sens : 29.41176470588235

数据集划分 random_state：12 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.4722222222222222
  mcc : -0.0768610838355672
  sens : 35.294117647058826
  spec : 57.89473684210527
数据集划分 random_state：13 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.16724840200141816
  sens : 35.714285714285715
  spec : 77.27272727272727
数据集划分 random_state：14 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5
  mcc : -0.05250181790691809
  sens : 33.33333333333333
  spec : 61.904761904761905
数据集划分 random_state：15 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.09607689228305229
  sens : 33.33333333333333
  spec : 75.0
数据集划分 random_state：16 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.0851256530758

数据集划分 random_state：15 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.2401922307076307
  sens : 41.66666666666667
  spec : 79.16666666666666
数据集划分 random_state：16 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6666666666666666
  mcc : 0.28210976821983985
  sens : 61.53846153846154
  spec : 69.56521739130434
数据集划分 random_state：17 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : -0.04003203845127179
  sens : 30.0
  spec : 65.38461538461539
数据集划分 random_state：18 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : 0.35641684448545224
  sens : 36.36363636363637
  spec : 85.71428571428571
数据集划分 random_state：19 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.15

...

数据集划分 random_state：19 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.08512565307587487
  sens : 38.46153846153847
  spec : 69.56521739130434
数据集划分 random_state：20 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.2138089935299395
  sens : 58.333333333333336
  spec : 66.66666666666666
数据集划分 random_state：21 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5555555555555556
  mcc : 0.12921914767618442
  sens : 44.44444444444444
  spec : 66.66666666666666
数据集划分 random_state：22 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 15, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.23570226039551584
  sens : 72.72727272727273
  spec : 60.0
数据集划分 random_state：23 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.18

数据集划分 random_state：28 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 15, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.05725025740766715
  sens : 30.76923076923077
  spec : 73.91304347826086
数据集划分 random_state：29 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.6944444444444444
  mcc : 0.3535839999514189
  sens : 64.28571428571429
  spec : 72.72727272727273
数据集划分 random_state：30 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 15, 'random_state': 0}
  accuracy : 0.4722222222222222
  mcc : -0.24618298195866545
  sens : 14.285714285714285
  spec : 68.18181818181817
以上为IDR__16时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 15, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.1336306209562122
  sens : 40.0
  spec : 71.42857142857143
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'learnin

数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.13957263155977065
  sens : 46.666666666666664
  spec : 66.66666666666666
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 15, 'random_state': 0}
  accuracy : 0.5277777777777778
  mcc : 0.051298917604257706
  sens : 47.05882352941176
  spec : 57.89473684210527
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 15, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.5040302036234264
  sens : 30.0
  spec : 93.75
数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5
  mcc : -0.0125
  sens : 43.75
  spec : 55.00000000000001
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.6111111111111112
  mcc : 0.14838816536047483
  sens : 54.54545454545454
  spec : 64

数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.4444444444444444
  mcc : -0.22776189843157396
  sens : 18.181818181818183
  spec : 56.00000000000001
数据集划分 random_state：5 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.4444444444444444
  mcc : -0.12059518353961064
  sens : 31.57894736842105
  spec : 58.82352941176471
数据集划分 random_state：6 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 15, 'random_state': 0}
  accuracy : 0.5
  mcc : 0.0
  sens : 50.0
  spec : 50.0
数据集划分 random_state：7 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.4722222222222222
  mcc : -0.019298915726909713
  sens : 50.0
  spec : 46.42857142857143
数据集划分 random_state：8 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5277777777777778
  mcc : -0.03876376661011099
  sens : 28.57142857142857
  spec : 68.1

数据集划分 random_state：8 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.05655768446177828
  sens : 21.428571428571427
  spec : 81.81818181818183
数据集划分 random_state：9 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.5833333333333334
  mcc : 0.2475368857441686
  sens : 42.10526315789473
  spec : 76.47058823529412
数据集划分 random_state：10 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 15, 'random_state': 0}
  accuracy : 0.6388888888888888
  mcc : 0.34426518632954817
  sens : 37.5
  spec : 85.0
数据集划分 random_state：11 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.5277777777777778
  mcc : 0.15240015240022858
  sens : 16.666666666666664
  spec : 88.88888888888889
数据集划分 random_state：12 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.5
  mcc : -0.02919836612898059
  sens : 29.

In [5]:
for k in range(21):
    IDR_acc = [i[0] for i in all_21_IDR_length_evaluate[k]]
    print("IDR_" + str(k+1) + "__max_acc_all_evaluate：" + str(all_21_IDR_length_evaluate[k][IDR_acc.index(np.max(IDR_acc))]))

IDR_1__max_acc_all_evaluate：[0.7222222222222222, 0.48801455452776477, 53.333333333333336, 85.71428571428571]
IDR_2__max_acc_all_evaluate：[0.75, 0.48281334215771954, 75.0, 75.0]
IDR_3__max_acc_all_evaluate：[0.7777777777777778, 0.4924344283300851, 84.61538461538461, 73.91304347826086]
IDR_4__max_acc_all_evaluate：[0.7222222222222222, 0.3697641828266701, 81.81818181818183, 68.0]
IDR_5__max_acc_all_evaluate：[0.75, 0.5744269579035527, 50.0, 90.9090909090909]
IDR_6__max_acc_all_evaluate：[0.75, 0.6017055648544889, 56.25, 90.0]
IDR_7__max_acc_all_evaluate：[0.7777777777777778, 0.3450327796711771, 37.5, 89.28571428571429]
IDR_8__max_acc_all_evaluate：[0.6666666666666666, 0.23145502494313785, 33.33333333333333, 83.33333333333334]
IDR_9__max_acc_all_evaluate：[0.8333333333333334, 0.7374757716174346, 54.54545454545454, 96.0]
IDR_10__max_acc_all_evaluate：[0.7222222222222222, 0.23719978133155237, 50.0, 78.57142857142857]
IDR_11__max_acc_all_evaluate：[0.7777777777777778, 0.4924344283300851, 84.6153846153

In [6]:
# 评估结果
all_IDR_avg_disorder_evaluate = []

# 将21个区域的平均disorder值一起作为特征
all_R_H_IDR_disorder =[]
all_R_H_IDR_disorder.extend(disorder_R_H[0])
all_R_H_IDR_disorder.extend(disorder_R_H[1])

# 设置超参数  max_depth：最大深度
param_dict = {"learning_rate": [0.01, 0.1],"max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 3)}
for r in range(0,31):
    one_random_state = []
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_R_H_IDR_disorder,input_lable,train_size=0.8,random_state = r)
    # 定义模型
    model = xgb.XGBClassifier()
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 最优的模型参数
    params = model.best_params_ 
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    
    one_random_state.append(accuracy)
    one_random_state.append(mcc)
    one_random_state.append(sens)
    one_random_state.append(spec)  
    all_IDR_avg_disorder_evaluate.append(one_random_state)
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("最优的模型参数是：" + str(params))
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.75
  mcc : 0.5345224838248488
  sens : 60.0
  spec : 85.71428571428571
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.7222222222222222
  mcc : 0.4705886038624533
  sens : 64.70588235294117
  spec : 78.94736842105263
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.01, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.9166666666666666
  mcc : 0.9176629354822471
  sens : 85.0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 5, 'random_state': 0}
  accuracy : 0.75
  mcc : 0.4681130931876265
  sens : 81.25
  spec : 70.0
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'learning_rate': 0.1, 'max_depth': 8, 'random_state': 0}
  accuracy : 0.7777777777777778
  mcc : 0.4763636363636364
  sens : 63.63636363636363
  spec : 84.0
数据集划分 random_state：5 时，
Evaluate